# 1. Import libraries

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 2. Read the data

In [3]:
project_dir = r"E:\1. Projects\ML-internship\customer-churn-prediction\Research"
data_dir = "data" 

In [4]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(project_dir, data_dir, file_name)

    return pd.read_csv(file_path)

In [5]:
data = get_data('churn_modelling')

In [6]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


```
-- The dataset has 10002 rows and 14 features
-- Columns 'Geography', 'Age', 'Hascrcard' and 'isactivemember' has one missing values
```

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10002 non-null  int64  
 1   CustomerId       10002 non-null  int64  
 2   Surname          10002 non-null  object 
 3   CreditScore      10002 non-null  int64  
 4   Geography        10001 non-null  object 
 5   Gender           10002 non-null  object 
 6   Age              10001 non-null  float64
 7   Tenure           10002 non-null  int64  
 8   Balance          10002 non-null  float64
 9   NumOfProducts    10002 non-null  int64  
 10  HasCrCard        10001 non-null  float64
 11  IsActiveMember   10001 non-null  float64
 12  EstimatedSalary  10002 non-null  float64
 13  Exited           10002 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 1.1+ MB


# 3. Some prelims analalysis

In [9]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


In [10]:
data.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                float64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard          float64
IsActiveMember     float64
EstimatedSalary    float64
Exited               int64
dtype: object

In [13]:
# Check for duplicates

data.duplicated().sum()

np.int64(2)

In [17]:
(
    data
    .loc[data.duplicated(keep= False)]
)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9998,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9999,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10000,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0
10001,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


In [18]:
# Some data cleaning process

def clean_data(df):
    return(
        df
        .dropna()
        .drop_duplicates()
        .rename(columns = str.lower)
    )


In [19]:
cleaned_data = clean_data(data)

In [20]:
cleaned_data.shape

(9996, 14)

In [22]:
cleaned_data.columns

Index(['rownumber', 'customerid', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard',
       'isactivemember', 'estimatedsalary', 'exited'],
      dtype='object')

In [28]:
num_col = ['rownumber', 'customerid', 'creditscore', 'age', 'tenure', 'balance', 'numofproducts', 'isactivemember', 'estimatedsalary']
cat_col = ['surname', 'geography', 'gender', 'hascrcard', 'exited']

In [30]:
cleaned_data.head()

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
5,6,15574012,Chu,645,Spain,Male,44.0,8,113755.78,2,1.0,0.0,149756.71,1


In [31]:
for col in cleaned_data.columns:
    print(f"Number of unique values in {col}: {cleaned_data[col].nunique()}")

Number of unique values in rownumber: 9996
Number of unique values in customerid: 9996
Number of unique values in surname: 2932
Number of unique values in creditscore: 460
Number of unique values in geography: 3
Number of unique values in gender: 2
Number of unique values in age: 73
Number of unique values in tenure: 11
Number of unique values in balance: 6379
Number of unique values in numofproducts: 4
Number of unique values in hascrcard: 2
Number of unique values in isactivemember: 2
Number of unique values in estimatedsalary: 9995
Number of unique values in exited: 2


# 4 Split the data

In [37]:
x = cleaned_data.drop(columns = 'exited')
y = cleaned_data.exited.copy()

In [38]:
x_, x_test, y_, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_, y_, test_size=0.2, random_state=42)

In [39]:
print(f"x_train_shape is {x_train.shape}, y_train_shape is {y_train.shape}")
print(f"x_val_shape is {x_val.shape}, y_val_shape is {y_val.shape}")
print(f"x_test_shape is {x_test.shape}, y_test_shape is {y_test.shape}")

x_train_shape is (6396, 13), y_train_shape is (6396,)
x_val_shape is (1600, 13), y_val_shape is (1600,)
x_test_shape is (2000, 13), y_test_shape is (2000,)


# 5. Export the subsets


In [40]:
def export_data(x, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(project_dir, data_dir, file_name)

    x.join(y).to_csv(file_path,index = False)

    return pd.read_csv(file_path).head()

In [41]:
export_data(x_train, y_train, "train")


,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,9470,15674573,Gearhart,713,France,Female,25.0,4,121172.97,1,1.0,1.0,56268.98,0
1,4798,15666173,Chidumaga,793,Germany,Female,32.0,1,96408.98,1,1.0,1.0,138191.81,0
2,943,15804586,Lin,376,France,Female,46.0,6,0.00,1,1.0,0.0,157333.69,1
3,9466,15815259,Fang,835,France,Female,56.0,2,0.00,2,1.0,1.0,39820.13,0
4,4147,15698246,Gordon,658,France,Female,24.0,2,0.00,2,1.0,1.0,84694.49,0


In [42]:
export_data(x_val, y_val, "val")

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,2861,15705890,Nebechukwu,674,France,Male,45.0,7,142072.02,1,1.0,0.0,37013.29,0
1,3631,15782456,Odili,656,France,Male,46.0,9,143267.14,2,0.0,0.0,193099.43,0
2,4467,15595160,Renwick,413,Spain,Male,35.0,2,0.00,2,1.0,1.0,60972.84,0
3,4335,15754649,Fang,705,Spain,Female,57.0,3,0.00,2,1.0,1.0,34134.14,0
4,9212,15605531,Daly,457,Spain,Female,38.0,6,0.00,2,1.0,0.0,173219.09,0


In [43]:
export_data(x_test, y_test, "test")

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,1083,15663438,Andrejew,688,Spain,Male,36.0,0,89772.30,1,1.0,0.0,177383.68,1
1,4341,15692631,Bogdanova,577,Spain,Female,44.0,8,115557.00,1,0.0,1.0,127506.76,0
2,8971,15636105,Chung,758,Spain,Male,61.0,2,0.00,2,1.0,1.0,43982.41,0
3,5461,15668894,Abramova,661,Germany,Male,41.0,5,122552.48,2,0.0,1.0,120646.40,0
4,2412,15739502,Amaechi,549,Germany,Female,31.0,9,135020.21,2,1.0,1.0,23343.18,0
